
## Objectives
- Interacting atomically with two Dapps
- Interacting with various types of on-chain exchanges as a trader
- Hands-on experience using Uniswap V3 as an LP

## Prerequisites 
- Metamask extension should be installed on the browser, if not, install Metamask from [here](https://metamask.io/).
- Sign up for an HTTP web3 provider service such as Infura. The function of a provider is to answer queries about the state of the contract that can be later used to make a trade and post transactions containing the swaps onto the testnet.

## Interacting atomically with two Dapps


#### Hotel and Flight contracts

- [Hotel Reservation Contract](https://sepolia.etherscan.io/address/0xE0FF10388e0556dCe87bcC589F59604c51C75DF0): `0xE0FF10388e0556dCe87bcC589F59604c51C75DF0`
- [Flight Reservation Contract](https://sepolia.etherscan.io/address/0xAD49e85a227605B105076664b6A44e08F842AC02): `0xAD49e85a227605B105076664b6A44e08F842AC02` 

#### Hotel reservation

Assume you are going on a vacation and need to book travel and accommodation. We have created two contracts mimicking a decentralized application for booking accommodation and travel. Let us go through its main ``bookRoom()`` function. 

```solidity
    function bookRoom() payable external {
        require(msg.value >= room_price, "Please pay full price"); 
        require(booked_rooms < total_rooms, "No more rooms available");

        hasReserved[current_iteration_ID][msg.sender] = true;
        booked_rooms += 1;

        if (msg.value > room_price) {
            uint256 refund = msg.value - room_price;
            (bool success, ) = payable(msg.sender).call{value: refund}("");
            require(success, "Refund transfer failed");
        }
    }
```

- Initial few checks ensure the payment is sufficient and rooms are available at the hotel
- The state of the contract is updated to make sure the room is booked in the name of the wallet initiating the transaction calling the function
- The last part calculates and disburses a refund if the transaction caller sent more money than the price `rooms.price = 0.00001 ether`.
- Note that no part of the contract allows for cancellation of the reservation; this Dapp corresponds to a Non-refundable reservation

You will now book accommodation for the trip. 
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. 
- Click the `bookRoom()` tab. 
- Set the amount of Wei to send as a payment. The required `payableAmount` is `0.00001` ETH which is 10000000000000 wei; You can use the Unit Converter [here](https://www.alchemy.com/gwei-calculator). 
- You can send more if you want, and the contract will refund it within the same transaction.
- Check the status of your reservation by querying `CheckReservations` with your public key

#### Flight reservation

Now that we have booked the accommodation, let us book a flight. The flight reservation contract looks similar to the hotel reservation contract with ``bookFlight()`` replacing ``bookRoom()``. Perform the same steps as the hotel reservation contract to book a flight ticket. Check the status of the transaction on Etherscan.

Notice that the **transaction will fail for some of you since there aren't enough flight tickets available** for all class members. This leads to a situation where your hotel reservation is practically useless since there is no way to travel. 


Question: 

Is it possible to avoid such a situation: *Book hotel only if the flight is available* and *Book flight only if the hotel is available*? In other words, we want the trip reservation to be atomic that either book both travel and accommodation or book none. Notice that the Dapps are independent of one another - think of them as two separate travel websites. 

## Atomic transaction

- [Atomic transactions](https://sepolia.etherscan.io/address/0xFBF85875Fd9389b223c1CC0CbD5FC51E4B3A2A3b): 0xFBF85875Fd9389b223c1CC0CbD5FC51E4B3A2A3b

We can perform such transactions easily on blockchains. Note that an ethereum transaction either succeeds or fails and reverts back. Thus, we need to combine ``bookRoom()`` and ``bookFlight()`` into a single transaction. 

We can do this by launching a smart contract whose one function call will call two of the above hotel and flight contracts. Let us quickly go through the ``bookTrip`` function:

```solidity
    /**
     * @dev Books a complete trip (hotel + flight) atomically.
     * 1. Calculates the combined cost of the hotel and the flight.
     * 2. Validates that the user has sent enough Ether.
     * 3. Calls the external hotel and flight contracts to perform the bookings.
     * 4. Refunds any excess Ether back to the user.
     */
    function bookTrip() payable external {
        // Calculate the total cost required for both reservations
        uint256 total_cost = hotel_reservation.getRoomPrice() + flight_reservation.getFlightPrice();

        // Check if the transferred value is sufficient
        if (msg.value < total_cost) {
            revert InsufficientPayment(msg.value, total_cost);
        }
        
        // Execute the booking calls to external contracts passing the specific required amounts
        hotel_reservation.bookRoomTo{value: hotel_reservation.getRoomPrice()}(msg.sender);
        flight_reservation.bookFlightTo{value: flight_reservation.getFlightPrice()}(msg.sender);
        
        // Calculate refund amount
        uint256 refund = msg.value - total_cost;
        address payable refund_address = payable(msg.sender);

        // Process refund if there is excess payment
        if (refund > 0) {
            (bool success, ) = payable(refund_address).call{value: refund}("");
            require(success, "Refund transfer failed");
        }
    }
```

- The function calls both the hotel and flight contract instances to get the price of the room and flight ticket
- It checks if the payment sent with the transaction is sufficient to cover both bookings
- And requires there are available rooms and flight tickets at the same time 


I have refreshed both the hotel and flight contracts, and you will now use the atomic transaction contract to book both atomically:
- Go to the contract on Etherscan, click the write contract tab, and connect your wallet. 
- Click the `bookTrip`. 
- Set the amount of ETH to send as a payment. The required total payment is at least `0.00002 ETH`; anything extra will be refunded. 
- Check the status of the transaction on Etherscan

The transaction will succeed for some of you and fail for the rest. Check the status of your booking on the respective contracts. If the atomic transaction failed, you should not have confirmed reservations on both the hotel and flight contracts. 


# Trading on decentralized exchange applications

Decentralized exchanges can be categorized into two broad categories:
- Dapps mimicking traditional centralized exchanges and implementing an order book
- Dapps running DeFi native exchange formats such as AMMs
In this lab, we will interact with one exchange of each of the above types through their web app.

## Trade on an order book exchange


Order book exchange enables trading by running sequential trades on an order book. The order book contains *bids* from agents who want to buy an asset, and *asks* from agents who want to sell the asset. A trade occurs when a bid matches an ask. One can perform two primary types of orders on such an exchange. 
- A *market order* is an order that executes the trade at the best available bid or asks.
- A *limit order* that sets a bid or ask on an order book that is executed at a price within limits set by the order.

In this lab, we will interact with dYdX V4, an order book DEX implemented on the dYdX Chain testnet. 

- Open [dYdX V4 testnet app](https://v4.testnet.dydx.exchange/#/trade/ETH-USD)
- Click on the top-right corner to connect your metamask wallet: `Sign in` -> `send request` -> `confirm`
- After the connection, click *Deposit funds* (or *deposit*)

We will now be performing a market order. Please follow the steps below: 
- Select market order on the right
- Trade 10 USD for ETH - Click on `sell` and set 10 USD
- If there is not enough ETH-USD liquidity, try to switch to another trading pair such as ADA-USD, etc. 
- Notice that you can choose the `Leverage` as well (here, we use the default)
- Click *Place Market Order* 
- Observe the change of Equity and Buying Power on the top-right
- Turn to Fills, check the information such as type, side, amount, price

We will now perform a limit order. Please follow the steps below:
- Select limit order on the right
- Observe the Price of USD for both *bids* and *asks*
- Set a price between the lowest of asks and the highest of bids as the Limit Price (fill into the Limit Price)
- Trade 10 USD for ETH: click on sell and set 10 USD
- Set other parameters as you want (here, we use the default)
- Click *Place Limit Order* 
- Turn to Orders to see the order in process, observe the change of *asks*
- After finishing, turn to Fills to check the information

## Trade on an AMM

We will be contrasting the above trading experience with Uniswap. Uniswap is an AMM; its implementation is lightweight by design; hence, it can be implemented entirely on an L1 blockchain such as Ethereum. As a contract with dYdX V4, trading on Uniswap does not require an account since it interacts directly with the wallet. We have created a trading pair to exchange USTUSD-USTETH. Let us perform a simple USTUSD-USTETH trade on Uniswap.

- Open the Uniswap app and select swap at [https://app.uniswap.org/#/swap](https://app.uniswap.org/#/swap)
- Connect your metamask wallet on the top right, and swith to `Sepolia` testnet by clicking on the setting icon. 
- Check `USTUSD` and `USTETH` funds on metamask
- If metamask doesn't show the USTUSD/USTETH token; we need to import it (USTUSD: 0xc83B0efA5B3F13851DfA11de72EF6AFeF026730c, USTETH: 0x4E22e9951770b98d4f3B2BA6647f0f40A15A4057)
- Paste the USTUSD contract address on the from tab (0xc83B0efA5B3F13851DfA11de72EF6AFeF026730c)
- Paste the USTETH contract address on the to tab (0x4E22e9951770b98d4f3B2BA6647f0f40A15A4057)
- You would need to approve USTUSD tokens to be used by the Uniswap contract; click on approve and confirm on Metamask.
- Once approval is complete, swap 10 USTUSD by clicking on the swap tab and confirming the transaction on Metamask
- Click on swap and confirm the transaction on metamask
- Select the txid link (on the top-right Pending tab) on Uniswap and observe the trade transaction on Etherscan 
- The trade should show pending and should be successful in less than 30 sec
- Check USTUSD and USTETH funds on metamask again



Note that all interactions are visible through your metamask wallet since they occur on the Sepolia blockchain. AMMs like Uniswap also allow anyone to create their trading pair, enabling exchanges for long-tail assets such as USTUSD and USTETH.

*[Hint: If your wallet has high latency, you may need to change the Seperolia RPC endpoint in Metamask settings to another public RPC URL such as `https://1rpc.io/sepolia`. You can find all famous RPC endpoints at https://chainlist.org/chain/11155111 ]*


# Interacting with Uniswap as a Liquidity provider

Uniswap allows anyone to provide liquidity to any token pair. Depositing tokens in a token-pair's liquidity pool enables that entity to own a pool and get the exchange fee rewards. In this lab, we will provide liquidity to the USTUSD-USTETH liquidity pool on Uniswap. 

- Open Uniswap app for adding liquidity at [liquidity](https://app.uniswap.org/#/add)
- Click on add liquidity
- Select the first token in the pair as USTUSD (set address as: 0xe41Fa6BF04aAF0dD6E44d62b1A1Bd8209dc06f69)
- Select the second token in the pair as USTETH (set address as: 0x3d1544B6FecE99C62a0f14D32759eb2cCE183670)
- Set 10 USTUSD, which should automatically populate USTETH based on the current state of the bonding curve
- Approve spending USTUSD and USTETH
- Wait for the approval transaction to be included in the chain
- Click on supply to add liquidity to the pool
- Observe the transaction on Etherscan 
- It should state that an ERC-721 token was transferred to your account
- Observe the transaction on Etherscan, you will get the information of the NFT (`address` and `token ID`)
- This ERC721 token is an LP token for the USTUSD-USTETH liquidity pool
- You need to add this token to metamask to see it (under the `NFT` section in metamask) by entering the `address` and `token ID` obtained from Etherscan
- The position should appear on [https://app.uniswap.org/#/pool](https://app.uniswap.org/#/pool)

The LP token is like any other token; you can transfer it between accounts and have that account claim ownership of the liquidity pool. To check this, create a new wallet account as follows:
- On metamask, click on the 'my accounts' icon on the top right and select ``Create account``

Now we will transfer ownership of the liquidity pool by sending LP tokens to your second account
- Change your account in your metamask to your original account
- Click on the `NFT` token, and `send`
- Paste the address of your second account, then Confirm the transaction
- Import the same `NFT` token on your second account (the address and token ID obtained from Etherscan)
- Go to Uniswap and check the status of your Liquidity positions on [https://app.uniswap.org/#/pool](https://app.uniswap.org/#/pool)
- Notice that it does not show any liquidity available for the first account
- Go to metamask and switch to the second account (click on connect)
- Observe that all liquidity is shown in the second account
